In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from dfply import *
import numpy as np
import re
from more_dfply import extract
from more_dfply import ifelse
from more_dfply import fix_names
import calendar
%matplotlib inline

In [3]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

In [ ]:
#!pip install pycountry_convert

In [ ]:
#!pip install --upgrade pycountry

# Bank VS. Country Death, Recovories, Cases

In [87]:
#population_data = pd.read_csv("./data/Population_Data.csv")
country_cases = pd.read_csv("./data/CountryCases.csv")
country_deaths = pd.read_csv("./data/CountryDeaths.csv")
country_recovery = pd.read_csv("./data/CountryRecoveries.csv")


# TODO 

1.) In country_cases, deaths, recovery, bin the Countries into North America, Asia, Austrailia, Europe, South America.. etc. The package pycountry will help classify the countries. 

2.) Take the end of each month like the 30th and the 31st. This will help see monthly changes. Do not Need Everyday in country data sets

3.) Clean the column names in all data sets if need be
 
    
4.) Country data sets the there needs to be 12 entries for each Region. Might be best to transform first and then use Pandas to     strip the day and filter by only 30's and 31's. THat way it will catch 2021 dates too. This can be done by using gather and spread code provided in module 6.

        Ex: North America   1/30/2020
                            2/30/2020
                            3/30/2020
                            ......etc
                            
   Will have to spread at the end so I can get one unique country. This will be my key for joining.
                         
 5.) Join all the data together. Make sure to join on countries. Should be around 210 total.
 
 6.) Bring into tableau and make graphs. Bar charts are good. Try to get lines as well (maybe do male for bar charts and female      for the lines)

# Cleaning Country Cases Data set

In [37]:
country_cases.head()

,Country,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2021-02-07,2021-02-08,2021-02-09,2021-02-10,2021-02-11,2021-02-12,2021-02-13,2021-02-14,2021-02-15,2021-02-16
0,United States,1,1,2,2,5,5,5,5,5,...,27007368,27097095,27192455,27287159,27392512,27492023,27575344,27640282,27694226,27756624
1,India,0,0,0,0,0,0,0,0,1,...,10838194,10847304,10858371,10871294,10880603,10892746,10904940,10916589,10925710,10937320
2,Brazil,0,0,0,0,0,0,0,0,0,...,9524640,9524640,9599565,9659167,9713909,9765455,9809754,9834513,9866710,9921981
3,United Kingdom,0,0,0,0,0,0,0,0,0,...,3957177,3971315,3983756,3996833,4010376,4025574,4038929,4049920,4059696,4070332
4,Russia,0,0,0,0,0,0,0,0,0,...,3923461,3939162,3953970,3968228,3983031,3997898,4012538,4026506,4040505,4053535


In [38]:
(country_cases >>  gather('Date',
                          'Cases',
                         columns_from('2020-01-22')))

,Country,Date,Cases
0,United States,2020-01-22,1
1,India,2020-01-22,0
2,Brazil,2020-01-22,0
3,United Kingdom,2020-01-22,0
4,Russia,2020-01-22,0
...,...,...,...
85059,Guam,2021-02-16,0
85060,Guernsey,2021-02-16,0
85061,Jersey,2021-02-16,0
85062,Palestine,2021-02-16,0


In [88]:
country_cases = (country_cases >> 
                   gather('Date',
                          'Cases',
                         columns_from('2020-01-22')))
country_cases

,Country,Date,Cases
0,United States,2020-01-22,1
1,India,2020-01-22,0
2,Brazil,2020-01-22,0
3,United Kingdom,2020-01-22,0
4,Russia,2020-01-22,0
...,...,...,...
85059,Guam,2021-02-16,0
85060,Guernsey,2021-02-16,0
85061,Jersey,2021-02-16,0
85062,Palestine,2021-02-16,0


In [89]:
def getdates(data):
    data['Date']= pd.to_datetime(data['Date'])
    data['Month'] = data.Date.dt.month
    data['Month_Name'] = data.Date.dt.month_name()
    data['Year'] = data.Date.dt.year
    data['Day'] = data.Date.dt.day
    data['Day_in_month'] = data.Date.dt.days_in_month
    return data

In [90]:
getdates(country_cases)

,Country,Date,Cases,Month,Month_Name,Year,Day,Day_in_month
0,United States,2020-01-22,1,1,January,2020,22,31
1,India,2020-01-22,0,1,January,2020,22,31
2,Brazil,2020-01-22,0,1,January,2020,22,31
3,United Kingdom,2020-01-22,0,1,January,2020,22,31
4,Russia,2020-01-22,0,1,January,2020,22,31
...,...,...,...,...,...,...,...,...
85059,Guam,2021-02-16,0,2,February,2021,16,28
85060,Guernsey,2021-02-16,0,2,February,2021,16,28
85061,Jersey,2021-02-16,0,2,February,2021,16,28
85062,Palestine,2021-02-16,0,2,February,2021,16,28


In [91]:
country_cases = (country_cases
>>filter_by(X.Day == X.Day_in_month))
country_cases

,Country,Date,Cases,Month,Month_Name,Year,Day,Day_in_month
1953,United States,2020-01-31,6,1,January,2020,31,31
1954,India,2020-01-31,1,1,January,2020,31,31
1955,Brazil,2020-01-31,0,1,January,2020,31,31
1956,United Kingdom,2020-01-31,2,1,January,2020,31,31
1957,Russia,2020-01-31,2,1,January,2020,31,31
...,...,...,...,...,...,...,...,...
81587,Guam,2021-01-31,0,1,January,2021,31,31
81588,Guernsey,2021-01-31,0,1,January,2021,31,31
81589,Jersey,2021-01-31,0,1,January,2021,31,31
81590,Palestine,2021-01-31,0,1,January,2021,31,31


In [92]:
country_cases['Date'] = country_cases['Date'].astype(str)

In [93]:
country_cases=(country_cases
 >> select(X.Country,X.Cases,X.Date)
 >> spread(X.Date,X.Cases)
)
country_cases

,Country,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31
0,Afghanistan,0,1,175,2127,15208,31507,36665,38159,39268,41425,46274,51526,55023
1,Albania,0,0,243,773,1137,2535,5276,9513,13649,20875,38182,58316,78127
2,Algeria,0,1,716,4006,9394,13907,30394,44494,51530,57942,83199,99610,107339
3,Andorra,0,0,376,745,764,855,925,1176,2050,4756,6745,8049,9937
4,Angola,0,0,7,27,86,284,1148,2654,4972,10805,15139,17553,19796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,2,16,212,270,328,355,558,1044,1094,1180,1347,1465,1817
213,West Bank and Gaza,0,0,119,344,448,2428,11837,22729,39899,53520,85647,138004,158962
214,Yemen,0,0,0,6,323,1158,1728,1958,2034,2063,2077,2099,2121
215,Zambia,0,0,35,106,1057,1594,5963,12097,14759,16432,17647,20725,54217


In [94]:
country_cases = (country_cases
>> mutate(Country_clean = X.Country))
country_cases

,Country,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,Country_clean
0,Afghanistan,0,1,175,2127,15208,31507,36665,38159,39268,41425,46274,51526,55023,Afghanistan
1,Albania,0,0,243,773,1137,2535,5276,9513,13649,20875,38182,58316,78127,Albania
2,Algeria,0,1,716,4006,9394,13907,30394,44494,51530,57942,83199,99610,107339,Algeria
3,Andorra,0,0,376,745,764,855,925,1176,2050,4756,6745,8049,9937,Andorra
4,Angola,0,0,7,27,86,284,1148,2654,4972,10805,15139,17553,19796,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,2,16,212,270,328,355,558,1044,1094,1180,1347,1465,1817,Vietnam
213,West Bank and Gaza,0,0,119,344,448,2428,11837,22729,39899,53520,85647,138004,158962,West Bank and Gaza
214,Yemen,0,0,0,6,323,1158,1728,1958,2034,2063,2077,2099,2121,Yemen
215,Zambia,0,0,35,106,1057,1594,5963,12097,14759,16432,17647,20725,54217,Zambia


In [95]:
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
}

In [96]:
check_country = {'West Bank and Gaza': 'Palestine','Burma':'Myanmar','Kosovo':'Serbia','Congo (Kinshasa)':'Congo',
                 'Congo (Brazzaville)':'Congo','The Gambia':'Senegal','Diamond Princess':'Others','Cruise Ship':'Others',
                'East Timor':'Others','Reunion':'Others','Vatican City':'Italy','MS Zaandam':'Others',
                 'Channel Islands':'France','Curacao':'Others','North Ireland':'Ireland','Saint Barthelemy':'France',
                'Cote d\'Ivoire':'Liberia','Holy See':'Greece','Korea, South':'South Korea','Taiwan*':'Taiwan',
                'Timor-Leste':'Others'}

In [97]:
country_cases = (country_cases
>> mutate(Country_clean = X.Country_clean.replace(check_country)))

In [98]:
country_cases = (country_cases
>> filter_by(X.Country_clean != 'Others'))

In [99]:
countries_cases = np.array(country_cases.Country_clean)
countries_cases

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Myanmar',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'France',
       'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Congo', 'Congo',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland',
       'France', 'French Guiana', 'Gabon', 'Georgia', 'Germany', 'Ghana',
       'Gibraltar', 'Greece',

In [100]:
countries_binned_cases = [continents[country_alpha2_to_continent_code(country_name_to_country_alpha2(country))] for country in countries_cases]

In [101]:
country_cases['Country_Binned'] = countries_binned_cases

In [53]:
#change_column_name = {'2020-01-31':'January_2020','2020-02-29':'February_2020','2020-03-31':'March_2020',"2020-04-30":'April_2020',
#                      '2020-05-31':'May_2020',"2020-06-30":"June_2020","2020-07-31":'July_2020','2020-08-31':'August_2020',
#                      '2020-09-30':'September_2020','2020-10-31':'October_2020','2020-11-30':'November_2020',"2020-12-31":'December_2020',
#                     '2021-01-31':'January_2021'}

In [103]:
#country_cases = country_cases.rename(columns=change_column_name)
#country_cases

In [104]:
country_cases.columns = [str(col) + '_cases' for col in country_cases.columns]

In [105]:
country_cases_final = country_cases
country_cases_final

,Country_cases,2020-01-31_cases,2020-02-29_cases,2020-03-31_cases,2020-04-30_cases,2020-05-31_cases,2020-06-30_cases,2020-07-31_cases,2020-08-31_cases,2020-09-30_cases,2020-10-31_cases,2020-11-30_cases,2020-12-31_cases,2021-01-31_cases,Country_clean_cases,Country_Binned_cases
0,Afghanistan,0,1,175,2127,15208,31507,36665,38159,39268,41425,46274,51526,55023,Afghanistan,Asia
1,Albania,0,0,243,773,1137,2535,5276,9513,13649,20875,38182,58316,78127,Albania,Europe
2,Algeria,0,1,716,4006,9394,13907,30394,44494,51530,57942,83199,99610,107339,Algeria,Africa
3,Andorra,0,0,376,745,764,855,925,1176,2050,4756,6745,8049,9937,Andorra,Europe
4,Angola,0,0,7,27,86,284,1148,2654,4972,10805,15139,17553,19796,Angola,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,2,16,212,270,328,355,558,1044,1094,1180,1347,1465,1817,Vietnam,Asia
213,West Bank and Gaza,0,0,119,344,448,2428,11837,22729,39899,53520,85647,138004,158962,Palestine,Asia
214,Yemen,0,0,0,6,323,1158,1728,1958,2034,2063,2077,2099,2121,Yemen,Asia
215,Zambia,0,0,35,106,1057,1594,5963,12097,14759,16432,17647,20725,54217,Zambia,Africa


#  Cleaning Country Death Data

In [57]:
country_deaths.head()

,Country,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2021-02-07,2021-02-08,2021-02-09,2021-02-10,2021-02-11,2021-02-12,2021-02-13,2021-02-14,2021-02-15,2021-02-16
0,United States,0,0,0,0,0,0,0,0,0,...,468204,469786,472818,476100,479257,482142,484301,485384,486325,488081
1,India,0,0,0,0,0,0,0,0,0,...,155080,155158,155252,155360,155447,155550,155642,155732,155813,155913
2,Brazil,0,0,0,0,0,0,0,0,0,...,231534,231534,233520,234850,236201,237489,238532,239245,239773,240940
3,United Kingdom,0,0,0,0,0,0,0,0,0,...,112681,113014,114066,115068,115748,116507,117128,117387,117622,118421
4,Russia,0,0,0,0,0,0,0,0,0,...,75430,75828,76347,76873,77415,77911,78403,78825,79210,79659


In [106]:
country_deaths = (country_deaths >> 
                   gather('Date',
                          'Cases',
                         columns_from('2020-01-22')))
country_deaths

,Country,Date,Cases
0,United States,2020-01-22,0
1,India,2020-01-22,0
2,Brazil,2020-01-22,0
3,United Kingdom,2020-01-22,0
4,Russia,2020-01-22,0
...,...,...,...
85059,Guam,2021-02-16,0
85060,Guernsey,2021-02-16,0
85061,Jersey,2021-02-16,0
85062,Palestine,2021-02-16,0


In [107]:
getdates(country_deaths)

,Country,Date,Cases,Month,Month_Name,Year,Day,Day_in_month
0,United States,2020-01-22,0,1,January,2020,22,31
1,India,2020-01-22,0,1,January,2020,22,31
2,Brazil,2020-01-22,0,1,January,2020,22,31
3,United Kingdom,2020-01-22,0,1,January,2020,22,31
4,Russia,2020-01-22,0,1,January,2020,22,31
...,...,...,...,...,...,...,...,...
85059,Guam,2021-02-16,0,2,February,2021,16,28
85060,Guernsey,2021-02-16,0,2,February,2021,16,28
85061,Jersey,2021-02-16,0,2,February,2021,16,28
85062,Palestine,2021-02-16,0,2,February,2021,16,28


In [108]:
country_deaths = (country_deaths
>>filter_by(X.Day == X.Day_in_month))
country_deaths

,Country,Date,Cases,Month,Month_Name,Year,Day,Day_in_month
1953,United States,2020-01-31,0,1,January,2020,31,31
1954,India,2020-01-31,0,1,January,2020,31,31
1955,Brazil,2020-01-31,0,1,January,2020,31,31
1956,United Kingdom,2020-01-31,0,1,January,2020,31,31
1957,Russia,2020-01-31,0,1,January,2020,31,31
...,...,...,...,...,...,...,...,...
81587,Guam,2021-01-31,0,1,January,2021,31,31
81588,Guernsey,2021-01-31,0,1,January,2021,31,31
81589,Jersey,2021-01-31,0,1,January,2021,31,31
81590,Palestine,2021-01-31,0,1,January,2021,31,31


In [109]:
country_deaths['Date'] = country_deaths['Date'].astype(str)

In [110]:
country_deaths=(country_deaths
 >> select(X.Country,X.Cases,X.Date)
 >> spread(X.Date,X.Cases)
)
country_deaths

,Country,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31
0,Afghanistan,0,0,4,64,258,752,1284,1403,1460,1538,1795,2191,2400
1,Albania,0,0,15,31,33,62,157,284,387,509,810,1181,1380
2,Algeria,0,0,44,450,653,912,1210,1510,1736,1964,2431,2756,2891
3,Andorra,0,0,12,42,51,52,52,53,53,75,76,84,101
4,Angola,0,0,2,2,4,13,52,108,183,284,348,405,466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,0,0,0,0,0,0,3,34,35,35,35,35,35
213,West Bank and Gaza,0,0,1,2,3,8,82,152,311,483,732,1400,1833
214,Yemen,0,0,0,2,80,312,493,566,587,599,605,610,615
215,Zambia,0,0,0,3,7,24,151,288,332,349,357,388,763


In [111]:
country_deaths = (country_deaths
>> mutate(Country_clean = X.Country))
country_deaths

,Country,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,Country_clean
0,Afghanistan,0,0,4,64,258,752,1284,1403,1460,1538,1795,2191,2400,Afghanistan
1,Albania,0,0,15,31,33,62,157,284,387,509,810,1181,1380,Albania
2,Algeria,0,0,44,450,653,912,1210,1510,1736,1964,2431,2756,2891,Algeria
3,Andorra,0,0,12,42,51,52,52,53,53,75,76,84,101,Andorra
4,Angola,0,0,2,2,4,13,52,108,183,284,348,405,466,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,0,0,0,0,0,0,3,34,35,35,35,35,35,Vietnam
213,West Bank and Gaza,0,0,1,2,3,8,82,152,311,483,732,1400,1833,West Bank and Gaza
214,Yemen,0,0,0,2,80,312,493,566,587,599,605,610,615,Yemen
215,Zambia,0,0,0,3,7,24,151,288,332,349,357,388,763,Zambia


In [112]:
country_deaths = (country_deaths
>> mutate(Country_clean = X.Country_clean.replace(check_country)))

In [113]:
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
}

In [114]:
check_country = {'West Bank and Gaza': 'Palestine','Burma':'Myanmar','Kosovo':'Serbia','Congo (Kinshasa)':'Congo',
                 'Congo (Brazzaville)':'Congo','The Gambia':'Senegal','Diamond Princess':'Others','Cruise Ship':'Others',
                'East Timor':'Others','Reunion':'Others','Vatican City':'Italy','MS Zaandam':'Others',
                 'Channel Islands':'France','Curacao':'Others','North Ireland':'Ireland','Saint Barthelemy':'France',
                'Cote d\'Ivoire':'Liberia','Holy See':'Greece','Korea, South':'South Korea','Taiwan*':'Taiwan',
                'Timor-Leste':'Others'}

In [115]:
country_deaths = (country_deaths
>> filter_by(X.Country_clean != 'Others'))

In [116]:
countries_deaths = np.array(country_deaths.Country_clean)
countries_deaths

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Myanmar',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'France',
       'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Congo', 'Congo',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland',
       'France', 'French Guiana', 'Gabon', 'Georgia', 'Germany', 'Ghana',
       'Gibraltar', 'Greece',

In [117]:
countries_binned_deaths = [continents[country_alpha2_to_continent_code(country_name_to_country_alpha2(country))] for country in countries_deaths]

In [118]:
country_deaths['Country_Binned'] = countries_binned_deaths
country_deaths

,Country,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,Country_clean,Country_Binned
0,Afghanistan,0,0,4,64,258,752,1284,1403,1460,1538,1795,2191,2400,Afghanistan,Asia
1,Albania,0,0,15,31,33,62,157,284,387,509,810,1181,1380,Albania,Europe
2,Algeria,0,0,44,450,653,912,1210,1510,1736,1964,2431,2756,2891,Algeria,Africa
3,Andorra,0,0,12,42,51,52,52,53,53,75,76,84,101,Andorra,Europe
4,Angola,0,0,2,2,4,13,52,108,183,284,348,405,466,Angola,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,0,0,0,0,0,0,3,34,35,35,35,35,35,Vietnam,Asia
213,West Bank and Gaza,0,0,1,2,3,8,82,152,311,483,732,1400,1833,Palestine,Asia
214,Yemen,0,0,0,2,80,312,493,566,587,599,605,610,615,Yemen,Asia
215,Zambia,0,0,0,3,7,24,151,288,332,349,357,388,763,Zambia,Africa


In [119]:
#country_deaths = country_deaths.rename(columns=change_column_name)
#country_deaths

In [120]:
country_deaths.columns = [str(col) + '_deaths' for col in country_deaths.columns]

In [121]:
country_deaths_final = country_deaths
country_deaths_final

,Country_deaths,2020-01-31_deaths,2020-02-29_deaths,2020-03-31_deaths,2020-04-30_deaths,2020-05-31_deaths,2020-06-30_deaths,2020-07-31_deaths,2020-08-31_deaths,2020-09-30_deaths,2020-10-31_deaths,2020-11-30_deaths,2020-12-31_deaths,2021-01-31_deaths,Country_clean_deaths,Country_Binned_deaths
0,Afghanistan,0,0,4,64,258,752,1284,1403,1460,1538,1795,2191,2400,Afghanistan,Asia
1,Albania,0,0,15,31,33,62,157,284,387,509,810,1181,1380,Albania,Europe
2,Algeria,0,0,44,450,653,912,1210,1510,1736,1964,2431,2756,2891,Algeria,Africa
3,Andorra,0,0,12,42,51,52,52,53,53,75,76,84,101,Andorra,Europe
4,Angola,0,0,2,2,4,13,52,108,183,284,348,405,466,Angola,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,0,0,0,0,0,0,3,34,35,35,35,35,35,Vietnam,Asia
213,West Bank and Gaza,0,0,1,2,3,8,82,152,311,483,732,1400,1833,Palestine,Asia
214,Yemen,0,0,0,2,80,312,493,566,587,599,605,610,615,Yemen,Asia
215,Zambia,0,0,0,3,7,24,151,288,332,349,357,388,763,Zambia,Africa


# Cleaning Country Recoveries Data

In [76]:
country_recovery.head()

,Country,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2021-02-07,2021-02-08,2021-02-09,2021-02-10,2021-02-11,2021-02-12,2021-02-13,2021-02-14,2021-02-15,2021-02-16
0,United States,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,India,0,0,0,0,0,0,0,0,0,...,10534505,10548521,10561608,10573372,10589230,10600625,10611731,10621220,10633025,10644858
2,Brazil,0,0,0,0,0,0,0,0,0,...,8467982,8478818,8577207,8616282,8637050,8691664,8740445,8765048,8821887,8847264
3,United Kingdom,0,0,0,0,0,0,0,0,0,...,10270,10323,10427,10495,10598,10655,10850,10914,10959,11022
4,Russia,0,0,0,0,0,0,0,0,0,...,3418329,3434163,3455582,3477760,3499230,3519689,3538241,3553326,3567189,3584477


In [122]:
country_recovery= (country_recovery >> 
                   gather('Date',
                          'Cases',
                         columns_from('2020-01-22'),
                         add_id=True))
country_recovery

,Country,_ID,Date,Cases
0,United States,0,2020-01-22,0
1,India,1,2020-01-22,0
2,Brazil,2,2020-01-22,0
3,United Kingdom,3,2020-01-22,0
4,Russia,4,2020-01-22,0
...,...,...,...,...
85059,Guam,212,2021-02-16,0
85060,Guernsey,213,2021-02-16,0
85061,Jersey,214,2021-02-16,0
85062,Palestine,215,2021-02-16,0


In [123]:
getdates(country_recovery)

,Country,_ID,Date,Cases,Month,Month_Name,Year,Day,Day_in_month
0,United States,0,2020-01-22,0,1,January,2020,22,31
1,India,1,2020-01-22,0,1,January,2020,22,31
2,Brazil,2,2020-01-22,0,1,January,2020,22,31
3,United Kingdom,3,2020-01-22,0,1,January,2020,22,31
4,Russia,4,2020-01-22,0,1,January,2020,22,31
...,...,...,...,...,...,...,...,...,...
85059,Guam,212,2021-02-16,0,2,February,2021,16,28
85060,Guernsey,213,2021-02-16,0,2,February,2021,16,28
85061,Jersey,214,2021-02-16,0,2,February,2021,16,28
85062,Palestine,215,2021-02-16,0,2,February,2021,16,28


In [124]:
country_recovery = (country_recovery
>>filter_by(X.Day == X.Day_in_month))
country_recovery

,Country,_ID,Date,Cases,Month,Month_Name,Year,Day,Day_in_month
1953,United States,0,2020-01-31,0,1,January,2020,31,31
1954,India,1,2020-01-31,0,1,January,2020,31,31
1955,Brazil,2,2020-01-31,0,1,January,2020,31,31
1956,United Kingdom,3,2020-01-31,0,1,January,2020,31,31
1957,Russia,4,2020-01-31,0,1,January,2020,31,31
...,...,...,...,...,...,...,...,...,...
81587,Guam,212,2021-01-31,0,1,January,2021,31,31
81588,Guernsey,213,2021-01-31,0,1,January,2021,31,31
81589,Jersey,214,2021-01-31,0,1,January,2021,31,31
81590,Palestine,215,2021-01-31,0,1,January,2021,31,31


In [125]:
country_recovery['Date'] = country_recovery['Date'].astype(str)

In [126]:
country_recovery=(country_recovery
 >> select(X.Country,X.Cases,X.Date)
 >> spread(X.Date,X.Cases)
)
country_recovery

,Country,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31
0,Afghanistan,0,0,5,260,1328,14131,25509,29089,32789,34321,36831,41727,47679
1,Albania,0,0,52,470,872,1459,2952,5214,7847,11189,18849,33634,47424
2,Algeria,0,0,46,1779,5748,9897,20537,31244,36174,40201,53809,67127,73344
3,Andorra,0,0,10,468,694,799,807,908,1432,3475,5873,7432,9093
4,Angola,0,0,1,7,18,93,437,1071,1941,4523,7851,11044,18035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,0,16,58,219,279,335,373,707,1010,1062,1179,1325,1457
213,West Bank and Gaza,0,0,18,76,372,451,5077,15056,31743,46309,64987,117183,148993
214,Yemen,0,0,0,1,14,488,862,1131,1286,1366,1382,1394,1426
215,Zambia,0,0,0,55,779,1329,3803,11469,13959,15680,16998,18660,48000


In [127]:
country_recovery = (country_recovery
>> mutate(Country_clean = X.Country))
country_recovery

,Country,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,Country_clean
0,Afghanistan,0,0,5,260,1328,14131,25509,29089,32789,34321,36831,41727,47679,Afghanistan
1,Albania,0,0,52,470,872,1459,2952,5214,7847,11189,18849,33634,47424,Albania
2,Algeria,0,0,46,1779,5748,9897,20537,31244,36174,40201,53809,67127,73344,Algeria
3,Andorra,0,0,10,468,694,799,807,908,1432,3475,5873,7432,9093,Andorra
4,Angola,0,0,1,7,18,93,437,1071,1941,4523,7851,11044,18035,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,0,16,58,219,279,335,373,707,1010,1062,1179,1325,1457,Vietnam
213,West Bank and Gaza,0,0,18,76,372,451,5077,15056,31743,46309,64987,117183,148993,West Bank and Gaza
214,Yemen,0,0,0,1,14,488,862,1131,1286,1366,1382,1394,1426,Yemen
215,Zambia,0,0,0,55,779,1329,3803,11469,13959,15680,16998,18660,48000,Zambia


In [128]:
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
}

In [129]:
check_country = {'West Bank and Gaza': 'Palestine','Burma':'Myanmar','Kosovo':'Serbia','Congo (Kinshasa)':'Congo',
                 'Congo (Brazzaville)':'Congo','The Gambia':'Senegal','Diamond Princess':'Others','Cruise Ship':'Others',
                'East Timor':'Others','Reunion':'Others','Vatican City':'Italy','MS Zaandam':'Others',
                 'Channel Islands':'France','Curacao':'Others','North Ireland':'Ireland','Saint Barthelemy':'France',
                'Cote d\'Ivoire':'Liberia','Holy See':'Greece','Korea, South':'South Korea','Taiwan*':'Taiwan',
                'Timor-Leste':'Others'}

In [130]:
country_recovery = (country_recovery
>> mutate(Country_clean = X.Country_clean.replace(check_country)))

In [131]:
country_recovery = (country_recovery
>> filter_by(X.Country_clean != 'Others'))

In [132]:
countries_recovery = np.array(country_recovery.Country_clean)
countries_recovery

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Myanmar',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'France',
       'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Congo', 'Congo',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland',
       'France', 'French Guiana', 'Gabon', 'Georgia', 'Germany', 'Ghana',
       'Gibraltar', 'Greece',

In [133]:
countries_binned_recovery = [continents[country_alpha2_to_continent_code(country_name_to_country_alpha2(country))] for country in countries_recovery]

In [134]:
country_recovery['Country_Binned'] = countries_binned_recovery
country_recovery

,Country,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,Country_clean,Country_Binned
0,Afghanistan,0,0,5,260,1328,14131,25509,29089,32789,34321,36831,41727,47679,Afghanistan,Asia
1,Albania,0,0,52,470,872,1459,2952,5214,7847,11189,18849,33634,47424,Albania,Europe
2,Algeria,0,0,46,1779,5748,9897,20537,31244,36174,40201,53809,67127,73344,Algeria,Africa
3,Andorra,0,0,10,468,694,799,807,908,1432,3475,5873,7432,9093,Andorra,Europe
4,Angola,0,0,1,7,18,93,437,1071,1941,4523,7851,11044,18035,Angola,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,0,16,58,219,279,335,373,707,1010,1062,1179,1325,1457,Vietnam,Asia
213,West Bank and Gaza,0,0,18,76,372,451,5077,15056,31743,46309,64987,117183,148993,Palestine,Asia
214,Yemen,0,0,0,1,14,488,862,1131,1286,1366,1382,1394,1426,Yemen,Asia
215,Zambia,0,0,0,55,779,1329,3803,11469,13959,15680,16998,18660,48000,Zambia,Africa


In [135]:
#country_recovery = country_recovery.rename(columns=change_column_name)
#country_recovery

In [136]:
country_recovery.columns = [str(col) + '_recovery' for col in country_recovery.columns]

In [137]:
country_recovery_final = country_recovery
country_recovery_final

,Country_recovery,2020-01-31_recovery,2020-02-29_recovery,2020-03-31_recovery,2020-04-30_recovery,2020-05-31_recovery,2020-06-30_recovery,2020-07-31_recovery,2020-08-31_recovery,2020-09-30_recovery,2020-10-31_recovery,2020-11-30_recovery,2020-12-31_recovery,2021-01-31_recovery,Country_clean_recovery,Country_Binned_recovery
0,Afghanistan,0,0,5,260,1328,14131,25509,29089,32789,34321,36831,41727,47679,Afghanistan,Asia
1,Albania,0,0,52,470,872,1459,2952,5214,7847,11189,18849,33634,47424,Albania,Europe
2,Algeria,0,0,46,1779,5748,9897,20537,31244,36174,40201,53809,67127,73344,Algeria,Africa
3,Andorra,0,0,10,468,694,799,807,908,1432,3475,5873,7432,9093,Andorra,Europe
4,Angola,0,0,1,7,18,93,437,1071,1941,4523,7851,11044,18035,Angola,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Vietnam,0,16,58,219,279,335,373,707,1010,1062,1179,1325,1457,Vietnam,Asia
213,West Bank and Gaza,0,0,18,76,372,451,5077,15056,31743,46309,64987,117183,148993,Palestine,Asia
214,Yemen,0,0,0,1,14,488,862,1131,1286,1366,1382,1394,1426,Yemen,Asia
215,Zambia,0,0,0,55,779,1329,3803,11469,13959,15680,16998,18660,48000,Zambia,Africa


# Cleaning World Bank Data

In [142]:
population_data = pd.read_csv("./data/Country_Population.csv")

In [143]:
population_data

,Time,Time Code,Country Name,Country Code,"Population ages 10-14, female","Population ages 10-14, male","Population ages 20-24, female","Population ages 20-24, male","Population ages 30-34, female","Population ages 30-34, male","Population ages 40-44, female","Population ages 40-44, male","Population ages 50-54, female","Population ages 50-54, male","Population ages 65 and above, female","Population ages 65 and above, male"
0,2019,YR2019,Afghanistan,AFG,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433
1,2019,YR2019,Albania,ALB,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495
2,2019,YR2019,Algeria,DZA,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851
3,2019,YR2019,American Samoa,ASM,..,..,..,..,..,..,..,..,..,..,..,..
4,2019,YR2019,Andorra,AND,..,..,..,..,..,..,..,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,2019,YR2019,Sub-Saharan Africa,SSF,12.27660013,12.585673,9.080514597,9.197471473,6.620651994,6.586599051,4.521243309,4.434610537,2.938690838,2.776985998,3.350045818,2.656756086
259,2019,YR2019,Sub-Saharan Africa (excluding high income),SSA,12.2838295,12.59269482,9.082696445,9.19940806,6.620518169,6.586095821,4.517475039,4.430363664,2.933867043,2.771913878,3.337853433,2.64751823
260,2019,YR2019,Sub-Saharan Africa (IDA & IBRD countries),TSS,12.27660013,12.585673,9.080514597,9.197471473,6.620651994,6.586599051,4.521243309,4.434610537,2.938690838,2.776985998,3.350045818,2.656756086
261,2019,YR2019,Upper middle income,UMC,6.576188166,7.121284845,6.77936889,7.187641753,8.351645129,8.51610658,6.941164644,6.937517891,7.08906665,6.911829281,11.12653549,8.942610045


In [144]:
country_recovery_final.Country.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Channel Islands', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Congo (Brazzaville)',
       'Congo (Kinshasa)', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus',
       'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana',
       'Gabon', 'Georgia', 'Germany',

In [145]:
covid_countries = {'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Myanmar',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'France',
       'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Congo', 'Congo',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland',
       'France', 'French Guiana', 'Gabon', 'Georgia', 'Germany', 'Ghana',
       'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe',
       'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau',
       'Guyana', 'Haiti', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland',
       'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Ivory Coast', 'Jamaica', 'Japan', 'Jersey', 'Jordan',
       'Kazakhstan', 'Kenya', 'Serbia', 'Kuwait', 'Kyrgyzstan', 'Laos',
       'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya',
       'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macau', 'Macedonia',
       'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta',
       'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius',
       'Mayotte', 'Mexico', 'Micronesia', 'Moldova', 'Monaco', 'Mongolia',
       'Montenegro', 'Morocco', 'Mozambique', 'Namibia', 'Nepal',
       'Netherlands', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria',
       'Ireland', 'Norway', 'Oman', 'Pakistan', 'Palestine', 'Panama',
       'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland',
       'Portugal', 'Puerto Rico', 'Qatar', 'Romania', 'Russia', 'Rwanda',
       'France', 'Saint Kitts and Nevis', 'Saint Lucia',
       'Saint Vincent and the Grenadines', 'Samoa', 'San Marino',
       'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia',
       'Seychelles', 'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia',
       'Solomon Islands', 'Somalia', 'South Africa', 'South Korea',
       'South Sudan', 'Spain', 'Sri Lanka', 'St. Martin', 'Sudan',
       'Suriname', 'Sweden', 'Switzerland', 'Syria', 'Taiwan',
       'Tajikistan', 'Tanzania', 'Thailand', 'Senegal', 'Togo',
       'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Uganda', 'Ukraine',
       'United Arab Emirates', 'United Kingdom', 'United States',
       'Uruguay', 'Uzbekistan', 'Vanuatu', 'Italy', 'Venezuela',
       'Vietnam', 'Palestine', 'Yemen', 'Zambia', 'Zimbabwe'}

In [146]:
population_data = (population_data
 >> mutate(Country = X['Country Name'].isin(covid_countries)))
population_data

,Time,Time Code,Country Name,Country Code,"Population ages 10-14, female","Population ages 10-14, male","Population ages 20-24, female","Population ages 20-24, male","Population ages 30-34, female","Population ages 30-34, male","Population ages 40-44, female","Population ages 40-44, male","Population ages 50-54, female","Population ages 50-54, male","Population ages 65 and above, female","Population ages 65 and above, male",Country
0,2019,YR2019,Afghanistan,AFG,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,True
1,2019,YR2019,Albania,ALB,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,True
2,2019,YR2019,Algeria,DZA,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,True
3,2019,YR2019,American Samoa,ASM,..,..,..,..,..,..,..,..,..,..,..,..,False
4,2019,YR2019,Andorra,AND,..,..,..,..,..,..,..,..,..,..,..,..,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,2019,YR2019,Sub-Saharan Africa,SSF,12.27660013,12.585673,9.080514597,9.197471473,6.620651994,6.586599051,4.521243309,4.434610537,2.938690838,2.776985998,3.350045818,2.656756086,False
259,2019,YR2019,Sub-Saharan Africa (excluding high income),SSA,12.2838295,12.59269482,9.082696445,9.19940806,6.620518169,6.586095821,4.517475039,4.430363664,2.933867043,2.771913878,3.337853433,2.64751823,False
260,2019,YR2019,Sub-Saharan Africa (IDA & IBRD countries),TSS,12.27660013,12.585673,9.080514597,9.197471473,6.620651994,6.586599051,4.521243309,4.434610537,2.938690838,2.776985998,3.350045818,2.656756086,False
261,2019,YR2019,Upper middle income,UMC,6.576188166,7.121284845,6.77936889,7.187641753,8.351645129,8.51610658,6.941164644,6.937517891,7.08906665,6.911829281,11.12653549,8.942610045,False


In [147]:
(population_data
 >>select(X['Country Name'],X.Country)
 >>filter_by(X.Country == True))

,Country Name,Country
0,Afghanistan,True
1,Albania,True
2,Algeria,True
4,Andorra,True
5,Angola,True
...,...,...
208,Uzbekistan,True
209,Vanuatu,True
211,Vietnam,True
215,Zambia,True


In [148]:
population_data = (population_data
 >>filter_by(X.Country == True))
population_data

,Time,Time Code,Country Name,Country Code,"Population ages 10-14, female","Population ages 10-14, male","Population ages 20-24, female","Population ages 20-24, male","Population ages 30-34, female","Population ages 30-34, male","Population ages 40-44, female","Population ages 40-44, male","Population ages 50-54, female","Population ages 50-54, male","Population ages 65 and above, female","Population ages 65 and above, male",Country
0,2019,YR2019,Afghanistan,AFG,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,True
1,2019,YR2019,Albania,ALB,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,True
2,2019,YR2019,Algeria,DZA,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,True
4,2019,YR2019,Andorra,AND,..,..,..,..,..,..,..,..,..,..,..,..,True
5,2019,YR2019,Angola,AGO,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,2019,YR2019,Uzbekistan,UZB,8.350656219,8.832965068,8.503121809,8.915809469,8.916161105,9.014076253,6.269284659,6.12423037,4.930684564,4.510053303,5.175760675,4.011740707,True
209,2019,YR2019,Vanuatu,VUT,11.69273018,12.28179797,8.599604989,8.441310725,7.798084467,6.985095831,5.257568789,5.162525158,3.668055518,3.794446125,3.527366684,3.703679343,True
211,2019,YR2019,Vietnam,VNM,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,True
215,2019,YR2019,Zambia,ZMB,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,True


In [149]:
(population_data
>>select(~X.Time,~X['Time Code'],~X['Country Code']))

,Country Name,"Population ages 10-14, female","Population ages 10-14, male","Population ages 20-24, female","Population ages 20-24, male","Population ages 30-34, female","Population ages 30-34, male","Population ages 40-44, female","Population ages 40-44, male","Population ages 50-54, female","Population ages 50-54, male","Population ages 65 and above, female","Population ages 65 and above, male",Country
0,Afghanistan,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,True
1,Albania,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,True
2,Algeria,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,True
4,Andorra,..,..,..,..,..,..,..,..,..,..,..,..,True
5,Angola,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Uzbekistan,8.350656219,8.832965068,8.503121809,8.915809469,8.916161105,9.014076253,6.269284659,6.12423037,4.930684564,4.510053303,5.175760675,4.011740707,True
209,Vanuatu,11.69273018,12.28179797,8.599604989,8.441310725,7.798084467,6.985095831,5.257568789,5.162525158,3.668055518,3.794446125,3.527366684,3.703679343,True
211,Vietnam,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,True
215,Zambia,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,True


In [150]:
population_data = (population_data
>>select(~X.Time,~X['Time Code'],~X['Country Code']))
population_data

,Country Name,"Population ages 10-14, female","Population ages 10-14, male","Population ages 20-24, female","Population ages 20-24, male","Population ages 30-34, female","Population ages 30-34, male","Population ages 40-44, female","Population ages 40-44, male","Population ages 50-54, female","Population ages 50-54, male","Population ages 65 and above, female","Population ages 65 and above, male",Country
0,Afghanistan,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,True
1,Albania,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,True
2,Algeria,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,True
4,Andorra,..,..,..,..,..,..,..,..,..,..,..,..,True
5,Angola,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Uzbekistan,8.350656219,8.832965068,8.503121809,8.915809469,8.916161105,9.014076253,6.269284659,6.12423037,4.930684564,4.510053303,5.175760675,4.011740707,True
209,Vanuatu,11.69273018,12.28179797,8.599604989,8.441310725,7.798084467,6.985095831,5.257568789,5.162525158,3.668055518,3.794446125,3.527366684,3.703679343,True
211,Vietnam,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,True
215,Zambia,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,True


In [151]:
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
}

In [152]:
check_country = {'West Bank and Gaza': 'Palestine','Burma':'Myanmar','Kosovo':'Serbia','Congo (Kinshasa)':'Congo',
                 'Congo (Brazzaville)':'Congo','The Gambia':'Senegal','Diamond Princess':'Others','Cruise Ship':'Others',
                'East Timor':'Others','Reunion':'Others','Vatican City':'Italy','MS Zaandam':'Others',
                 'Channel Islands':'France','Curacao':'Others','North Ireland':'Ireland','Saint Barthelemy':'France',
                'Cote d\'Ivoire':'Liberia','Holy See':'Greece','Korea, South':'South Korea','Taiwan*':'Taiwan',
                'Timor-Leste':'Others'}

In [153]:
(population_data
>> filter_by(X['Country Name'] != 'Others'))

,Country Name,"Population ages 10-14, female","Population ages 10-14, male","Population ages 20-24, female","Population ages 20-24, male","Population ages 30-34, female","Population ages 30-34, male","Population ages 40-44, female","Population ages 40-44, male","Population ages 50-54, female","Population ages 50-54, male","Population ages 65 and above, female","Population ages 65 and above, male",Country
0,Afghanistan,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,True
1,Albania,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,True
2,Algeria,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,True
4,Andorra,..,..,..,..,..,..,..,..,..,..,..,..,True
5,Angola,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Uzbekistan,8.350656219,8.832965068,8.503121809,8.915809469,8.916161105,9.014076253,6.269284659,6.12423037,4.930684564,4.510053303,5.175760675,4.011740707,True
209,Vanuatu,11.69273018,12.28179797,8.599604989,8.441310725,7.798084467,6.985095831,5.257568789,5.162525158,3.668055518,3.794446125,3.527366684,3.703679343,True
211,Vietnam,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,True
215,Zambia,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,True


In [154]:
population_countries = np.array(population_data['Country Name'])
population_countries

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Cayman Islands', 'Central African Republic', 'Chad',
       'Chile', 'China', 'Colombia', 'Comoros', 'Costa Rica', 'Croatia',
       'Cuba', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Faroe Islands', 'Fiji', 'Finland', 'France', 'Gabon', 'Georgia',
       'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada',
       'Guam', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti',
       'Honduras', 'Hung

In [155]:
countries_binned_population = [continents[country_alpha2_to_continent_code(country_name_to_country_alpha2(country))] for country in population_countries]

In [156]:
population_data['Countries_Binned'] = countries_binned_population
population_data

,Country Name,"Population ages 10-14, female","Population ages 10-14, male","Population ages 20-24, female","Population ages 20-24, male","Population ages 30-34, female","Population ages 30-34, male","Population ages 40-44, female","Population ages 40-44, male","Population ages 50-54, female","Population ages 50-54, male","Population ages 65 and above, female","Population ages 65 and above, male",Country,Countries_Binned
0,Afghanistan,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,True,Asia
1,Albania,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,True,Europe
2,Algeria,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,True,Africa
4,Andorra,..,..,..,..,..,..,..,..,..,..,..,..,True,Europe
5,Angola,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,True,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Uzbekistan,8.350656219,8.832965068,8.503121809,8.915809469,8.916161105,9.014076253,6.269284659,6.12423037,4.930684564,4.510053303,5.175760675,4.011740707,True,Asia
209,Vanuatu,11.69273018,12.28179797,8.599604989,8.441310725,7.798084467,6.985095831,5.257568789,5.162525158,3.668055518,3.794446125,3.527366684,3.703679343,True,Australia
211,Vietnam,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,True,Asia
215,Zambia,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,True,Africa


In [157]:
population_data.columns = [col.replace('Population ', '') for col in population_data.columns]
population_data

,Country Name,"ages 10-14, female","ages 10-14, male","ages 20-24, female","ages 20-24, male","ages 30-34, female","ages 30-34, male","ages 40-44, female","ages 40-44, male","ages 50-54, female","ages 50-54, male","ages 65 and above, female","ages 65 and above, male",Country,Countries_Binned
0,Afghanistan,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,True,Asia
1,Albania,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,True,Europe
2,Algeria,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,True,Africa
4,Andorra,..,..,..,..,..,..,..,..,..,..,..,..,True,Europe
5,Angola,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,True,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Uzbekistan,8.350656219,8.832965068,8.503121809,8.915809469,8.916161105,9.014076253,6.269284659,6.12423037,4.930684564,4.510053303,5.175760675,4.011740707,True,Asia
209,Vanuatu,11.69273018,12.28179797,8.599604989,8.441310725,7.798084467,6.985095831,5.257568789,5.162525158,3.668055518,3.794446125,3.527366684,3.703679343,True,Australia
211,Vietnam,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,True,Asia
215,Zambia,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,True,Africa


In [158]:
population_data.columns = [col.replace(',', '') for col in population_data.columns]
population_data

,Country Name,ages 10-14 female,ages 10-14 male,ages 20-24 female,ages 20-24 male,ages 30-34 female,ages 30-34 male,ages 40-44 female,ages 40-44 male,ages 50-54 female,ages 50-54 male,ages 65 and above female,ages 65 and above male,Country,Countries_Binned
0,Afghanistan,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,True,Asia
1,Albania,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,True,Europe
2,Algeria,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,True,Africa
4,Andorra,..,..,..,..,..,..,..,..,..,..,..,..,True,Europe
5,Angola,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,True,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Uzbekistan,8.350656219,8.832965068,8.503121809,8.915809469,8.916161105,9.014076253,6.269284659,6.12423037,4.930684564,4.510053303,5.175760675,4.011740707,True,Asia
209,Vanuatu,11.69273018,12.28179797,8.599604989,8.441310725,7.798084467,6.985095831,5.257568789,5.162525158,3.668055518,3.794446125,3.527366684,3.703679343,True,Australia
211,Vietnam,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,True,Asia
215,Zambia,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,True,Africa


In [159]:
population_data.columns = [col.replace(' ', '_') for col in population_data.columns]
population_data

,Country_Name,ages_10-14_female_,ages_10-14_male_,ages_20-24_female_,ages_20-24_male,ages_30-34_female_,ages_30-34_male_,ages_40-44_female_,ages_40-44_male_,ages_50-54_female_,ages_50-54_male_,ages_65_and_above_female_,ages_65_and_above_male,Country,Countries_Binned
0,Afghanistan,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,True,Asia
1,Albania,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,True,Europe
2,Algeria,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,True,Africa
4,Andorra,..,..,..,..,..,..,..,..,..,..,..,..,True,Europe
5,Angola,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,True,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Uzbekistan,8.350656219,8.832965068,8.503121809,8.915809469,8.916161105,9.014076253,6.269284659,6.12423037,4.930684564,4.510053303,5.175760675,4.011740707,True,Asia
209,Vanuatu,11.69273018,12.28179797,8.599604989,8.441310725,7.798084467,6.985095831,5.257568789,5.162525158,3.668055518,3.794446125,3.527366684,3.703679343,True,Australia
211,Vietnam,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,True,Asia
215,Zambia,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,True,Africa


In [160]:
population_data_final = (population_data
>>select(~X.Country))
population_data_final

,Country_Name,ages_10-14_female_,ages_10-14_male_,ages_20-24_female_,ages_20-24_male,ages_30-34_female_,ages_30-34_male_,ages_40-44_female_,ages_40-44_male_,ages_50-54_female_,ages_50-54_male_,ages_65_and_above_female_,ages_65_and_above_male,Countries_Binned
0,Afghanistan,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,Asia
1,Albania,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,Europe
2,Algeria,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,Africa
4,Andorra,..,..,..,..,..,..,..,..,..,..,..,..,Europe
5,Angola,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Uzbekistan,8.350656219,8.832965068,8.503121809,8.915809469,8.916161105,9.014076253,6.269284659,6.12423037,4.930684564,4.510053303,5.175760675,4.011740707,Asia
209,Vanuatu,11.69273018,12.28179797,8.599604989,8.441310725,7.798084467,6.985095831,5.257568789,5.162525158,3.668055518,3.794446125,3.527366684,3.703679343,Australia
211,Vietnam,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,Asia
215,Zambia,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,Africa


# Joining All tables together
{country_cases_final, country_recovery_final, country_deaths_final, world bank data}

In [138]:
country_recovery_final = country_recovery_final.rename(columns={'Country_recovery':'Country'})
country_recovery_final.columns

Index(['Country', '2020-01-31_recovery', '2020-02-29_recovery',
       '2020-03-31_recovery', '2020-04-30_recovery', '2020-05-31_recovery',
       '2020-06-30_recovery', '2020-07-31_recovery', '2020-08-31_recovery',
       '2020-09-30_recovery', '2020-10-31_recovery', '2020-11-30_recovery',
       '2020-12-31_recovery', '2021-01-31_recovery', 'Country_clean_recovery',
       'Country_Binned_recovery'],
      dtype='object')

In [139]:
country_cases_final = country_cases_final.rename(columns={'Country_cases':'Country'})
country_cases_final.columns

Index(['Country', '2020-01-31_cases', '2020-02-29_cases', '2020-03-31_cases',
       '2020-04-30_cases', '2020-05-31_cases', '2020-06-30_cases',
       '2020-07-31_cases', '2020-08-31_cases', '2020-09-30_cases',
       '2020-10-31_cases', '2020-11-30_cases', '2020-12-31_cases',
       '2021-01-31_cases', 'Country_clean_cases', 'Country_Binned_cases'],
      dtype='object')

In [140]:
country_deaths_final = country_deaths_final.rename(columns={'Country_deaths':'Country'})
country_deaths_final.columns

Index(['Country', '2020-01-31_deaths', '2020-02-29_deaths',
       '2020-03-31_deaths', '2020-04-30_deaths', '2020-05-31_deaths',
       '2020-06-30_deaths', '2020-07-31_deaths', '2020-08-31_deaths',
       '2020-09-30_deaths', '2020-10-31_deaths', '2020-11-30_deaths',
       '2020-12-31_deaths', '2021-01-31_deaths', 'Country_clean_deaths',
       'Country_Binned_deaths'],
      dtype='object')

In [161]:
population_data_final = population_data_final.rename(columns={'Country_Name':'Country'})
population_data_final.columns

Index(['Country', 'ages_10-14_female_', 'ages_10-14_male_',
       'ages_20-24_female_', 'ages_20-24_male', 'ages_30-34_female_',
       'ages_30-34_male_', 'ages_40-44_female_', 'ages_40-44_male_',
       'ages_50-54_female_', 'ages_50-54_male_', 'ages_65_and_above_female_',
       'ages_65_and_above_male', 'Countries_Binned'],
      dtype='object')

In [162]:
country_cases_recovery = country_recovery_final >> left_join(country_cases_final, by = 'Country')
country_cases_recovery

,Country,2020-01-31_recovery,2020-02-29_recovery,2020-03-31_recovery,2020-04-30_recovery,2020-05-31_recovery,2020-06-30_recovery,2020-07-31_recovery,2020-08-31_recovery,2020-09-30_recovery,...,2020-06-30_cases,2020-07-31_cases,2020-08-31_cases,2020-09-30_cases,2020-10-31_cases,2020-11-30_cases,2020-12-31_cases,2021-01-31_cases,Country_clean_cases,Country_Binned_cases
0,Afghanistan,0,0,5,260,1328,14131,25509,29089,32789,...,31507,36665,38159,39268,41425,46274,51526,55023,Afghanistan,Asia
1,Albania,0,0,52,470,872,1459,2952,5214,7847,...,2535,5276,9513,13649,20875,38182,58316,78127,Albania,Europe
2,Algeria,0,0,46,1779,5748,9897,20537,31244,36174,...,13907,30394,44494,51530,57942,83199,99610,107339,Algeria,Africa
3,Andorra,0,0,10,468,694,799,807,908,1432,...,855,925,1176,2050,4756,6745,8049,9937,Andorra,Europe
4,Angola,0,0,1,7,18,93,437,1071,1941,...,284,1148,2654,4972,10805,15139,17553,19796,Angola,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Vietnam,0,16,58,219,279,335,373,707,1010,...,355,558,1044,1094,1180,1347,1465,1817,Vietnam,Asia
206,West Bank and Gaza,0,0,18,76,372,451,5077,15056,31743,...,2428,11837,22729,39899,53520,85647,138004,158962,Palestine,Asia
207,Yemen,0,0,0,1,14,488,862,1131,1286,...,1158,1728,1958,2034,2063,2077,2099,2121,Yemen,Asia
208,Zambia,0,0,0,55,779,1329,3803,11469,13959,...,1594,5963,12097,14759,16432,17647,20725,54217,Zambia,Africa


In [163]:
country_final_data = country_cases_recovery >> left_join(country_deaths_final, by = 'Country')
country_final_data.columns

Index(['Country', '2020-01-31_recovery', '2020-02-29_recovery',
       '2020-03-31_recovery', '2020-04-30_recovery', '2020-05-31_recovery',
       '2020-06-30_recovery', '2020-07-31_recovery', '2020-08-31_recovery',
       '2020-09-30_recovery', '2020-10-31_recovery', '2020-11-30_recovery',
       '2020-12-31_recovery', '2021-01-31_recovery', 'Country_clean_recovery',
       'Country_Binned_recovery', '2020-01-31_cases', '2020-02-29_cases',
       '2020-03-31_cases', '2020-04-30_cases', '2020-05-31_cases',
       '2020-06-30_cases', '2020-07-31_cases', '2020-08-31_cases',
       '2020-09-30_cases', '2020-10-31_cases', '2020-11-30_cases',
       '2020-12-31_cases', '2021-01-31_cases', 'Country_clean_cases',
       'Country_Binned_cases', '2020-01-31_deaths', '2020-02-29_deaths',
       '2020-03-31_deaths', '2020-04-30_deaths', '2020-05-31_deaths',
       '2020-06-30_deaths', '2020-07-31_deaths', '2020-08-31_deaths',
       '2020-09-30_deaths', '2020-10-31_deaths', '2020-11-30_deaths',
 

In [164]:
(country_final_data
>>select(~X.Country_clean_recovery,~X.Country_Binned_recovery,
          ~X.Country_clean_cases,~X.Country_Binned_cases))

,Country,2020-01-31_recovery,2020-02-29_recovery,2020-03-31_recovery,2020-04-30_recovery,2020-05-31_recovery,2020-06-30_recovery,2020-07-31_recovery,2020-08-31_recovery,2020-09-30_recovery,...,2020-06-30_deaths,2020-07-31_deaths,2020-08-31_deaths,2020-09-30_deaths,2020-10-31_deaths,2020-11-30_deaths,2020-12-31_deaths,2021-01-31_deaths,Country_clean_deaths,Country_Binned_deaths
0,Afghanistan,0,0,5,260,1328,14131,25509,29089,32789,...,752,1284,1403,1460,1538,1795,2191,2400,Afghanistan,Asia
1,Albania,0,0,52,470,872,1459,2952,5214,7847,...,62,157,284,387,509,810,1181,1380,Albania,Europe
2,Algeria,0,0,46,1779,5748,9897,20537,31244,36174,...,912,1210,1510,1736,1964,2431,2756,2891,Algeria,Africa
3,Andorra,0,0,10,468,694,799,807,908,1432,...,52,52,53,53,75,76,84,101,Andorra,Europe
4,Angola,0,0,1,7,18,93,437,1071,1941,...,13,52,108,183,284,348,405,466,Angola,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Vietnam,0,16,58,219,279,335,373,707,1010,...,0,3,34,35,35,35,35,35,Vietnam,Asia
206,West Bank and Gaza,0,0,18,76,372,451,5077,15056,31743,...,8,82,152,311,483,732,1400,1833,Palestine,Asia
207,Yemen,0,0,0,1,14,488,862,1131,1286,...,312,493,566,587,599,605,610,615,Yemen,Asia
208,Zambia,0,0,0,55,779,1329,3803,11469,13959,...,24,151,288,332,349,357,388,763,Zambia,Africa


In [165]:
country_final_data = (country_final_data
>>select(~X.Country_clean_recovery,~X.Country_Binned_recovery,
          ~X.Country_clean_cases,~X.Country_Binned_cases))
country_final_data.columns 

Index(['Country', '2020-01-31_recovery', '2020-02-29_recovery',
       '2020-03-31_recovery', '2020-04-30_recovery', '2020-05-31_recovery',
       '2020-06-30_recovery', '2020-07-31_recovery', '2020-08-31_recovery',
       '2020-09-30_recovery', '2020-10-31_recovery', '2020-11-30_recovery',
       '2020-12-31_recovery', '2021-01-31_recovery', '2020-01-31_cases',
       '2020-02-29_cases', '2020-03-31_cases', '2020-04-30_cases',
       '2020-05-31_cases', '2020-06-30_cases', '2020-07-31_cases',
       '2020-08-31_cases', '2020-09-30_cases', '2020-10-31_cases',
       '2020-11-30_cases', '2020-12-31_cases', '2021-01-31_cases',
       '2020-01-31_deaths', '2020-02-29_deaths', '2020-03-31_deaths',
       '2020-04-30_deaths', '2020-05-31_deaths', '2020-06-30_deaths',
       '2020-07-31_deaths', '2020-08-31_deaths', '2020-09-30_deaths',
       '2020-10-31_deaths', '2020-11-30_deaths', '2020-12-31_deaths',
       '2021-01-31_deaths', 'Country_clean_deaths', 'Country_Binned_deaths'],
      dty

In [166]:
country_final_data >> left_join(population_data_final, by = 'Country')


,Country,2020-01-31_recovery,2020-02-29_recovery,2020-03-31_recovery,2020-04-30_recovery,2020-05-31_recovery,2020-06-30_recovery,2020-07-31_recovery,2020-08-31_recovery,2020-09-30_recovery,...,ages_20-24_male,ages_30-34_female_,ages_30-34_male_,ages_40-44_female_,ages_40-44_male_,ages_50-54_female_,ages_50-54_male_,ages_65_and_above_female_,ages_65_and_above_male,Countries_Binned
0,Afghanistan,0,0,5,260,1328,14131,25509,29089,32789,...,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,Asia
1,Albania,0,0,52,470,872,1459,2952,5214,7847,...,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,Europe
2,Algeria,0,0,46,1779,5748,9897,20537,31244,36174,...,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,Africa
3,Andorra,0,0,10,468,694,799,807,908,1432,...,..,..,..,..,..,..,..,..,..,Europe
4,Angola,0,0,1,7,18,93,437,1071,1941,...,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Vietnam,0,16,58,219,279,335,373,707,1010,...,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,Asia
206,West Bank and Gaza,0,0,18,76,372,451,5077,15056,31743,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,Yemen,0,0,0,1,14,488,862,1131,1286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,Zambia,0,0,0,55,779,1329,3803,11469,13959,...,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,Africa


In [167]:
covid_population_final = country_final_data >> left_join(population_data_final, by = 'Country')
covid_population_final.columns

Index(['Country', '2020-01-31_recovery', '2020-02-29_recovery',
       '2020-03-31_recovery', '2020-04-30_recovery', '2020-05-31_recovery',
       '2020-06-30_recovery', '2020-07-31_recovery', '2020-08-31_recovery',
       '2020-09-30_recovery', '2020-10-31_recovery', '2020-11-30_recovery',
       '2020-12-31_recovery', '2021-01-31_recovery', '2020-01-31_cases',
       '2020-02-29_cases', '2020-03-31_cases', '2020-04-30_cases',
       '2020-05-31_cases', '2020-06-30_cases', '2020-07-31_cases',
       '2020-08-31_cases', '2020-09-30_cases', '2020-10-31_cases',
       '2020-11-30_cases', '2020-12-31_cases', '2021-01-31_cases',
       '2020-01-31_deaths', '2020-02-29_deaths', '2020-03-31_deaths',
       '2020-04-30_deaths', '2020-05-31_deaths', '2020-06-30_deaths',
       '2020-07-31_deaths', '2020-08-31_deaths', '2020-09-30_deaths',
       '2020-10-31_deaths', '2020-11-30_deaths', '2020-12-31_deaths',
       '2021-01-31_deaths', 'Country_clean_deaths', 'Country_Binned_deaths',
       'ag

In [168]:
covid_population_final = (covid_population_final
                         >> select(~X.Country_Binned_deaths,~X.Country_clean_deaths))
covid_population_final.columns

Index(['Country', '2020-01-31_recovery', '2020-02-29_recovery',
       '2020-03-31_recovery', '2020-04-30_recovery', '2020-05-31_recovery',
       '2020-06-30_recovery', '2020-07-31_recovery', '2020-08-31_recovery',
       '2020-09-30_recovery', '2020-10-31_recovery', '2020-11-30_recovery',
       '2020-12-31_recovery', '2021-01-31_recovery', '2020-01-31_cases',
       '2020-02-29_cases', '2020-03-31_cases', '2020-04-30_cases',
       '2020-05-31_cases', '2020-06-30_cases', '2020-07-31_cases',
       '2020-08-31_cases', '2020-09-30_cases', '2020-10-31_cases',
       '2020-11-30_cases', '2020-12-31_cases', '2021-01-31_cases',
       '2020-01-31_deaths', '2020-02-29_deaths', '2020-03-31_deaths',
       '2020-04-30_deaths', '2020-05-31_deaths', '2020-06-30_deaths',
       '2020-07-31_deaths', '2020-08-31_deaths', '2020-09-30_deaths',
       '2020-10-31_deaths', '2020-11-30_deaths', '2020-12-31_deaths',
       '2021-01-31_deaths', 'ages_10-14_female_', 'ages_10-14_male_',
       'ages_20-2

In [169]:
covid_population_final.columns = covid_population_final.columns.str.replace('\_$','')
covid_population_final.columns

/home/vn6415dw/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


Index(['Country', '2020-01-31_recovery', '2020-02-29_recovery',
       '2020-03-31_recovery', '2020-04-30_recovery', '2020-05-31_recovery',
       '2020-06-30_recovery', '2020-07-31_recovery', '2020-08-31_recovery',
       '2020-09-30_recovery', '2020-10-31_recovery', '2020-11-30_recovery',
       '2020-12-31_recovery', '2021-01-31_recovery', '2020-01-31_cases',
       '2020-02-29_cases', '2020-03-31_cases', '2020-04-30_cases',
       '2020-05-31_cases', '2020-06-30_cases', '2020-07-31_cases',
       '2020-08-31_cases', '2020-09-30_cases', '2020-10-31_cases',
       '2020-11-30_cases', '2020-12-31_cases', '2021-01-31_cases',
       '2020-01-31_deaths', '2020-02-29_deaths', '2020-03-31_deaths',
       '2020-04-30_deaths', '2020-05-31_deaths', '2020-06-30_deaths',
       '2020-07-31_deaths', '2020-08-31_deaths', '2020-09-30_deaths',
       '2020-10-31_deaths', '2020-11-30_deaths', '2020-12-31_deaths',
       '2021-01-31_deaths', 'ages_10-14_female', 'ages_10-14_male',
       'ages_20-24_

# Exporting data into CSV's

In [170]:
country_final_data.to_csv('Covid_Data_Final.csv')

In [171]:
country_cases_final.to_csv('Covid_Cases_Final.csv')

In [172]:
country_recovery_final.to_csv('Covid_Recovery_Final.csv')

In [173]:
country_deaths_final.to_csv('Covid_Deaths_Final.csv')

In [174]:
covid_population_final.to_csv('Covid_Population_combined.csv')

# Stacking combined date columns for more use in visualizations

In [207]:
combined = pd.read_csv("./Covid_Population_combined.csv")

In [208]:
combined

,Unnamed: 0,Country,2020-01-31_recovery,2020-02-29_recovery,2020-03-31_recovery,2020-04-30_recovery,2020-05-31_recovery,2020-06-30_recovery,2020-07-31_recovery,2020-08-31_recovery,...,ages_20-24_male,ages_30-34_female,ages_30-34_male,ages_40-44_female,ages_40-44_male,ages_50-54_female,ages_50-54_male,ages_65_and_above_female,ages_65_and_above_male,Countries_Binned
0,0,Afghanistan,0,0,5,260,1328,14131,25509,29089,...,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,2.620193463,2.612504517,2.882233877,2.363156433,Asia
1,1,Albania,0,0,52,470,872,1459,2952,5214,...,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,6.903901203,6.233360717,15.04269769,13.39221495,Europe
2,2,Algeria,0,0,46,1779,5748,9897,20537,31244,...,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,4.723239327,4.592930301,6.750311265,6.359306851,Africa
3,3,Andorra,0,0,10,468,694,799,807,908,...,..,..,..,..,..,..,..,..,..,Europe
4,4,Angola,0,0,1,7,18,93,437,1071,...,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,2.458228963,2.330429662,2.41000291,1.984219658,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,205,Vietnam,0,16,58,219,279,335,373,707,...,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,6.036047128,5.937062785,9.156091451,5.945905735,Asia
206,206,West Bank and Gaza,0,0,18,76,372,451,5077,15056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,207,Yemen,0,0,0,1,14,488,862,1131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,208,Zambia,0,0,0,55,779,1329,3803,11469,...,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,2.359029122,2.262593668,2.549729314,1.67226531,Africa


In [177]:
list(combined.columns)

['Unnamed: 0',
 'Country',
 '2020-01-31_recovery',
 '2020-02-29_recovery',
 '2020-03-31_recovery',
 '2020-04-30_recovery',
 '2020-05-31_recovery',
 '2020-06-30_recovery',
 '2020-07-31_recovery',
 '2020-08-31_recovery',
 '2020-09-30_recovery',
 '2020-10-31_recovery',
 '2020-11-30_recovery',
 '2020-12-31_recovery',
 '2021-01-31_recovery',
 '2020-01-31_cases',
 '2020-02-29_cases',
 '2020-03-31_cases',
 '2020-04-30_cases',
 '2020-05-31_cases',
 '2020-06-30_cases',
 '2020-07-31_cases',
 '2020-08-31_cases',
 '2020-09-30_cases',
 '2020-10-31_cases',
 '2020-11-30_cases',
 '2020-12-31_cases',
 '2021-01-31_cases',
 '2020-01-31_deaths',
 '2020-02-29_deaths',
 '2020-03-31_deaths',
 '2020-04-30_deaths',
 '2020-05-31_deaths',
 '2020-06-30_deaths',
 '2020-07-31_deaths',
 '2020-08-31_deaths',
 '2020-09-30_deaths',
 '2020-10-31_deaths',
 '2020-11-30_deaths',
 '2020-12-31_deaths',
 '2021-01-31_deaths',
 'ages_10-14_female',
 'ages_10-14_male',
 'ages_20-24_female',
 'ages_20-24_male',
 'ages_30-34_femal

In [199]:
recovery_months = ['2020-01-31_recovery',
 '2020-02-29_recovery',
 '2020-03-31_recovery',
 '2020-04-30_recovery',
 '2020-05-31_recovery',
 '2020-06-30_recovery',
 '2020-07-31_recovery',
 '2020-08-31_recovery',
 '2020-09-30_recovery',
 '2020-10-31_recovery',
 '2020-11-30_recovery',
 '2020-12-31_recovery',
 '2021-01-31_recovery']

In [200]:
cases_months = ['2020-01-31_cases',
 '2020-02-29_cases',
 '2020-03-31_cases',
 '2020-04-30_cases',
 '2020-05-31_cases',
 '2020-06-30_cases',
 '2020-07-31_cases',
 '2020-08-31_cases',
 '2020-09-30_cases',
 '2020-10-31_cases',
 '2020-11-30_cases',
 '2020-12-31_cases',
 '2021-01-31_cases']

In [201]:
death_months = ['2020-01-31_deaths',
 '2020-02-29_deaths',
 '2020-03-31_deaths',
 '2020-04-30_deaths',
 '2020-05-31_deaths',
 '2020-06-30_deaths',
 '2020-07-31_deaths',
 '2020-08-31_deaths',
 '2020-09-30_deaths',
 '2020-10-31_deaths',
 '2020-11-30_deaths',
 '2020-12-31_deaths',
 '2021-01-31_deaths']

In [209]:
combined = (combined 
            >> gather('Month_Recovery','Recovery',recovery_months)
            >> gather('Month_Cases','Cases',cases_months)
            >> gather('Month_Death','Deaths',death_months))
combined

,Unnamed: 0,Country,ages_10-14_female,ages_10-14_male,ages_20-24_female,ages_20-24_male,ages_30-34_female,ages_30-34_male,ages_40-44_female,ages_40-44_male,...,ages_50-54_male,ages_65_and_above_female,ages_65_and_above_male,Countries_Binned,Month_Recovery,Recovery,Month_Cases,Cases,Month_Death,Deaths
0,0,Afghanistan,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,...,2.612504517,2.882233877,2.363156433,Asia,2020-01-31_recovery,0,2020-01-31_cases,0,2020-01-31_deaths,0
1,1,Albania,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,...,6.233360717,15.04269769,13.39221495,Europe,2020-01-31_recovery,0,2020-01-31_cases,0,2020-01-31_deaths,0
2,2,Algeria,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,...,4.592930301,6.750311265,6.359306851,Africa,2020-01-31_recovery,0,2020-01-31_cases,0,2020-01-31_deaths,0
3,3,Andorra,..,..,..,..,..,..,..,..,...,..,..,..,Europe,2020-01-31_recovery,0,2020-01-31_cases,0,2020-01-31_deaths,0
4,4,Angola,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,...,2.330429662,2.41000291,1.984219658,Africa,2020-01-31_recovery,0,2020-01-31_cases,0,2020-01-31_deaths,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461365,205,Vietnam,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,...,5.937062785,9.156091451,5.945905735,Asia,2021-01-31_recovery,1457,2021-01-31_cases,1817,2021-01-31_deaths,35
461366,206,West Bank and Gaza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-01-31_recovery,148993,2021-01-31_cases,158962,2021-01-31_deaths,1833
461367,207,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-01-31_recovery,1426,2021-01-31_cases,2121,2021-01-31_deaths,615
461368,208,Zambia,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,...,2.262593668,2.549729314,1.67226531,Africa,2021-01-31_recovery,48000,2021-01-31_cases,54217,2021-01-31_deaths,763


In [210]:
combined['Month_Recovery'] = combined['Month_Recovery'].str.replace('_recovery','')

In [211]:
combined['Month_Cases'] = combined['Month_Cases'].str.replace('_cases','')

In [212]:
combined['Month_Death'] = combined['Month_Death'].str.replace('_deaths','')

In [213]:
combined

,Unnamed: 0,Country,ages_10-14_female,ages_10-14_male,ages_20-24_female,ages_20-24_male,ages_30-34_female,ages_30-34_male,ages_40-44_female,ages_40-44_male,...,ages_50-54_male,ages_65_and_above_female,ages_65_and_above_male,Countries_Binned,Month_Recovery,Recovery,Month_Cases,Cases,Month_Death,Deaths
0,0,Afghanistan,13.51315874,13.38952152,9.961009138,9.988106849,6.14755378,6.430479033,4.097513887,4.272471397,...,2.612504517,2.882233877,2.363156433,Asia,2020-01-31,0,2020-01-31,0,2020-01-31,0
1,1,Albania,5.478358395,6.127823778,8.085760271,8.261469813,6.886434608,8.088029261,5.617101139,5.194012756,...,6.233360717,15.04269769,13.39221495,Europe,2020-01-31,0,2020-01-31,0,2020-01-31,0
2,2,Algeria,8.324689951,8.489353354,7.144027983,7.28309575,8.681971149,8.619346155,6.789647731,6.677523232,...,4.592930301,6.750311265,6.359306851,Africa,2020-01-31,0,2020-01-31,0,2020-01-31,0
3,3,Andorra,..,..,..,..,..,..,..,..,...,..,..,..,Europe,2020-01-31,0,2020-01-31,0,2020-01-31,0
4,4,Angola,13.09818339,13.24566067,8.76425638,8.740961051,6.258883419,6.132921904,4.114095813,3.961275036,...,2.330429662,2.41000291,1.984219658,Africa,2020-01-31,0,2020-01-31,0,2020-01-31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461365,205,Vietnam,6.863436374,7.601626638,7.207407394,7.607674235,8.516370557,8.787290126,7.168194594,7.223801317,...,5.937062785,9.156091451,5.945905735,Asia,2021-01-31,1457,2021-01-31,1817,2021-01-31,35
461366,206,West Bank and Gaza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-01-31,148993,2021-01-31,158962,2021-01-31,1833
461367,207,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-01-31,1426,2021-01-31,2121,2021-01-31,615
461368,208,Zambia,13.06806967,13.50012563,9.415501751,9.43025918,6.572686082,6.378099637,4.358392605,4.31836208,...,2.262593668,2.549729314,1.67226531,Africa,2021-01-31,48000,2021-01-31,54217,2021-01-31,763


In [188]:
combined.to_csv('Covid_Population_restacked.csv')